In [3]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
import requests
from creds import google_api_key
import os
import pandas as pd

In [4]:
partysummary = os.path.join('output','PartySummary.csv')

In [5]:
df = pd.read_csv(partysummary)

In [6]:
df.head()

,Party,Total Contestants,Won,Second,Third,Party Votes,Valid Votes %,Valid Votes % (Contested),AC Leads,AC Second,AC Third
0,Bharatiya Janta Party,428,282,54,78,171657549,31.3%,40%,1883,550,601
1,Indian National Congress,464,44,224,66,106938242,19.5%,22.6%,550,1748,531
2,All India Anna Dravida Munnetra Kazhagam,40,37,1,2,18115825,3.3%,44.4%,222,16,24
3,All India Trinamool Congress,131,34,3,7,21259681,3.9%,15.4%,215,55,90
4,Biju Janata Dal,21,20,1,0,9491497,1.7%,44.8%,116,26,5


In [7]:
urls = []
for x in range(0,9):
    urls.append('https://www.google.com/search?source=lnms&tbm=nws&q='+ df['Party'][x].replace(' ','+'))

In [8]:
all_list = []

for url in urls:
    webpage = requests.get(url)
    soup = bs(webpage.text, 'lxml')
    results = soup.find_all('h3', class_ = 'r')
    
    news_headlines = []
    for x in range(len(results)):
        news_headlines.append((results[x].text))
    all_list.append(news_headlines)

In [9]:
mydict = {}

for x in range(len(all_list)):
    mydict[df['Party'][x]] = all_list[x]

In [10]:
mydict

{'Bharatiya Janta Party': ['9 attacks in 5 years: Arvind Kejriwal blames Bharatiya Janata Party',
  "'Mahagathbandhan' isn't a challenge for Bharatiya Janata Party ...",
  'Daler Mehndi joins Bharatiya Janata Party',
  'India election phase 5: Latest updates',
  "India's BJP releases manifesto before elections",
  'Disgruntled MP Udit Raj quits Bharatiya Janata Party, joins Congress',
  "India's Ruling Party to Get Majority With Allies, Leader Says",
  "'Coalition government making attempts to muzzle media'",
  'Lok Sabha Elections 2019: Bollywood Actor Sunny Deol Joins ...',
  'Potential allies in pole position'],
 'Indian National Congress': ["Congress writes to EC; demands action over PM Modi's 'Rajiv ...",
  'No, this video does not show an Indian National Congress ...',
  'Star wars in Indian National Congress',
  'Kangana Ranaut attacks Congress on Mumbai poll day: We were ...',
  'What Indian Elections Could Mean for Higher Ed',
  'Congress Manifesto Raises Terrifying Questions 

## Sentiment Analysis using Google Language AI/Machine Learning 

In [14]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'creds.json'

In [31]:
sentiment_list = []
newdict = {}

In [37]:
client = language.LanguageServiceClient()

for key, value in mydict.items():
    for headlines in value:
        document = types.Document(content=headlines, type=enums.Document.Type.PLAIN_TEXT)
        sentiment = client.analyze_sentiment(document=document).document_sentiment
        sentiment_list.append(sentiment)
    newdict[key] = sentiment_list
    sentiment_list = []

In [39]:
newdict

{'Bharatiya Janta Party': [magnitude: 0.800000011920929
  score: -0.800000011920929, , magnitude: 0.699999988079071
  score: 0.699999988079071, , , magnitude: 0.4000000059604645
  score: -0.4000000059604645, , magnitude: 0.699999988079071
  score: -0.699999988079071, magnitude: 0.30000001192092896
  score: 0.30000001192092896, magnitude: 0.10000000149011612
  score: 0.10000000149011612, magnitude: 0.800000011920929
  score: -0.800000011920929, , magnitude: 0.699999988079071
  score: 0.699999988079071, , , magnitude: 0.4000000059604645
  score: -0.4000000059604645, , magnitude: 0.699999988079071
  score: -0.699999988079071, magnitude: 0.30000001192092896
  score: 0.30000001192092896, magnitude: 0.10000000149011612
  score: 0.10000000149011612, magnitude: 0.800000011920929
  score: -0.800000011920929, , magnitude: 0.699999988079071
  score: 0.699999988079071, , , magnitude: 0.4000000059604645
  score: -0.4000000059604645, , magnitude: 0.699999988079071
  score: -0.699999988079071, magnit

In [20]:
for x in range(0,10):
    print(f'{news_headlines[x]}: {sentiment_list[x]}')

TMC goons killed party worker: CPI(M): magnitude: 0.6000000238418579
score: -0.6000000238418579

India's main opposition poses no threat to Modi: magnitude: 0.10000000149011612
score: 0.10000000149011612

Bomb attacks leave Kallyot tense: magnitude: 0.800000011920929
score: 0.800000011920929

Complaint Against BJP's Locket Chatterjee For Threatening Bengal ...: magnitude: 0.4000000059604645
score: -0.4000000059604645

Political songs, music videos flood media in Bengal: magnitude: 0.10000000149011612
score: -0.10000000149011612

Centre must be pro-active after Fani: CPI-M: magnitude: 0.5
score: 0.5

Potential allies in pole position: magnitude: 0.10000000149011612
score: 0.10000000149011612

What Indian Elections Could Mean for Higher Ed: magnitude: 0.30000001192092896
score: 0.30000001192092896

Lok Sabha elections 2019- 'It's unfortunate that Rahul Gandhi ...: magnitude: 0.20000000298023224
score: -0.20000000298023224

CPI(M) candidate releases manifesto: 
